# Time series prediction using LSTM

## Dataset
- Daily minimum temperatures in Melbourne, Australia, 1981-1990
- Units: Degrees Celcius

## Question 1

### Load the data
- load daily-minimum-temperatures-in-me.csv

In [1]:
# if you are using google colab
from google.colab import files
files.upload()

Saving daily-minimum-temperatures-in-me.csv to daily-minimum-temperatures-in-me.csv


{'daily-minimum-temperatures-in-me.csv': b'Date,Temperature\n1981-01-01,20.7\n1981-01-02,17.9\n1981-01-03,18.8\n1981-01-04,14.6\n1981-01-05,15.8\n1981-01-06,15.8\n1981-01-07,15.8\n1981-01-08,17.4\n1981-01-09,21.8\n1981-01-10,20\n1981-01-11,16.2\n1981-01-12,13.3\n1981-01-13,16.7\n1981-01-14,21.5\n1981-01-15,25\n1981-01-16,20.7\n1981-01-17,20.6\n1981-01-18,24.8\n1981-01-19,17.7\n1981-01-20,15.5\n1981-01-21,18.2\n1981-01-22,12.1\n1981-01-23,14.4\n1981-01-24,16\n1981-01-25,16.5\n1981-01-26,18.7\n1981-01-27,19.4\n1981-01-28,17.2\n1981-01-29,15.5\n1981-01-30,15.1\n1981-01-31,15.4\n1981-02-01,15.3\n1981-02-02,18.8\n1981-02-03,21.9\n1981-02-04,19.9\n1981-02-05,16.6\n1981-02-06,16.8\n1981-02-07,14.6\n1981-02-08,17.1\n1981-02-09,25\n1981-02-10,15\n1981-02-11,13.7\n1981-02-12,13.9\n1981-02-13,18.3\n1981-02-14,22\n1981-02-15,22.1\n1981-02-16,21.2\n1981-02-17,18.4\n1981-02-18,16.6\n1981-02-19,16.1\n1981-02-20,15.7\n1981-02-21,16.6\n1981-02-22,16.5\n1981-02-23,14.4\n1981-02-24,14.4\n1981-02-25,18.5\

In [2]:
import pandas as pd
import numpy as np 

In [3]:
data = pd.read_csv('daily-minimum-temperatures-in-me.csv')

### Print the data
- First 5 rows only

In [4]:
data.head(5)

Date  Temperature
0  1981-01-01         20.7
1  1981-01-02         17.9
2  1981-01-03         18.8
3  1981-01-04         14.6
4  1981-01-05         15.8

### Set index column
- Set the DataFrame index using "Date" column

In [6]:
data.set_index('Date',inplace=True)

In [7]:
data.head()

Temperature
Date                   
1981-01-01         20.7
1981-01-02         17.9
1981-01-03         18.8
1981-01-04         14.6
1981-01-05         15.8

## Question 2

### Check the total count of null values in the data

In [9]:
data.isnull().any()

Temperature    False
dtype: bool

In [10]:
data.isna().sum().sum()

0

### Drop all the rows with null values
- use dropna() method

In [11]:
data.dropna()

Temperature
Date                   
1981-01-01         20.7
1981-01-02         17.9
1981-01-03         18.8
1981-01-04         14.6
1981-01-05         15.8
...                 ...
1990-12-27         14.0
1990-12-28         13.6
1990-12-29         13.5
1990-12-30         15.7
1990-12-31         13.0

[3650 rows x 1 columns]

## Question 3

### Describe your dataframe
- use describe() method

In [12]:
data.describe()

Temperature
count  3650.000000
mean     11.177753
std       4.071837
min       0.000000
25%       8.300000
50%      11.000000
75%      14.000000
max      26.300000

### Plot the dataframe
- use plot() method

In [20]:
import matplotlib.pyplot as plt
data.plot();

### Get representation of the distribution of data using histogram
- use hist() method

In [19]:
data.hist();

### Check minimum and maximum temperature
- use min() and max() methods

In [17]:
print ('minimum temprature = ',data.min())

minimum temprature =  Temperature    0.0
dtype: float64


In [18]:
print ('maximum temprature = ',data.max())

maximum temprature =  Temperature    26.3
dtype: float64


## Question 4

### Split the dataset into training and testing
- training data: 75%
- testing data: 25%

In [21]:
train_data, test_data= np.split(data, [int(.75 *len(data))])

In [23]:
print (train_data.shape)
print (test_data.shape)

(2737, 1)
(913, 1)


### Normalize the data
- normalize temperature values using MinMaxScaler
- feature_range: 0-1
- fit MinMaxScaler on train data and transform it
  - use fit_transform() method
- transform test data using MinMaxScaler object
  - use transform() method

In [34]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))


In [35]:
train_data_norm = scaler.fit_transform(train_data)
test_data_norm = scaler.transform(test_data)

In [38]:
print('Range of train_data',train_data_norm.min(),'-',train_data_norm.max())

Range of train_data 0.0 - 1.0


In [39]:
print('Range of test_data',test_data_norm.min(),'-',test_data_norm.max())

Range of test_data 0.019011406844106463 - 0.9087452471482889


### Print scaled train and test data
- print only first 5 values

In [40]:
print(train_data_norm[:5])

[[0.78707224]
 [0.68060837]
 [0.7148289 ]
 [0.55513308]
 [0.60076046]]


In [41]:
print(test_data_norm[:5])

[[0.22813688]
 [0.22053232]
 [0.18631179]
 [0.19011407]
 [0.31939163]]


## Question 5

### Create sequential data
Map the temperature at a particular day 'd' to the temperature at day t
'd+n', where n is any number you define.

For example: to map temperatures of consecutive days, use d+1, i.e. lookback = 1
- Below function takes data as input and returns sequential data.
- X as features and y as labels

In [43]:
import numpy as np
def create_seq_data(data, lookback=1):
  X, y = list(), list()
  
  for day in range((len(data)-lookback)):
    X.append(data[day: day+lookback, 0])
    y.append(data[day+lookback, 0])
  return np.array(X), np.array(y)

### Use the above function to create train and test sets
- Get X_train and y_train
- Get X_test and y_test
- lookback = 1

In [44]:
X_train,y_train = create_seq_data(train_data_norm,lookback=1)
X_test,y_test = create_seq_data(test_data_norm,lookback=1)

In [49]:
print(X_train)
print(y_train)

2736
[0.68060837 0.7148289  0.55513308 ... 0.25475285 0.35741445 0.35361217]


In [50]:
print(X_test[:5])
print(y_test[:5])

[[0.22813688]
 [0.22053232]
 [0.18631179]
 [0.19011407]
 [0.31939163]]
[0.22053232 0.18631179 0.19011407 0.31939163 0.46768061]


### Reshape the data
- Reshape X_train and X_test
- use np.reshape() method
- this is to be done to make the data ready for input in model

In [61]:
X_train_rshp = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [66]:
X_test_rshp = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

### Print shape of data
- print shape of X_train and X_test

In [68]:
print(X_train_rshp.shape)
print(X_test_rshp.shape)

(2736, 1, 1)
(912, 1, 1)


## Question 6

### Define sequential model
### Define sequential model
- initialize a Sequential model
- add an LSTM layer
  - units: 10
  - stateful: True
  - pass a `batch_input_shape` argument to your layer
  - batch_input_shape: (batch_size, look_back, 1)
    - batch_size: 1
    - look_back: 1
- add Dense layer with 1 neuron

In [91]:
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

In [92]:
import tensorflow
tensorflow.keras.backend.clear_session()

In [93]:
model = Sequential()
model.add(LSTM(units = 10, stateful=True, batch_input_shape=(1, 1, 1)))
model.add(Dense(50,activation='relu'))
model.add(Dense(1))

## Question 7

### compile your model
- loss: mean_squared_error
- optimizer: adam

In [94]:
model.compile(loss='mean_squared_error', optimizer='adam')

### summarize your model
- use summary() method

In [95]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (1, 10)                   480       
_________________________________________________________________
dense (Dense)                (1, 50)                   550       
_________________________________________________________________
dense_1 (Dense)              (1, 1)                    51        
Total params: 1,081
Trainable params: 1,081
Non-trainable params: 0
_________________________________________________________________


## Question 8

### fit your model
- epochs: 50
- batch_size: 1

In [96]:
model.fit(X_train_rshp,y_train,batch_size=1,epochs=50)

Epoch 1/50
2736/2736 [==============================] - 5s 2ms/step - loss: 0.0133
Epoch 2/50
2736/2736 [==============================] - 5s 2ms/step - loss: 0.0105
Epoch 3/50
2736/2736 [==============================] - 5s 2ms/step - loss: 0.0104
Epoch 4/50
2736/2736 [==============================] - 5s 2ms/step - loss: 0.0103
Epoch 5/50
2736/2736 [==============================] - 4s 2ms/step - loss: 0.0102
Epoch 6/50
2736/2736 [==============================] - 4s 2ms/step - loss: 0.0102
Epoch 7/50
2736/2736 [==============================] - 5s 2ms/step - loss: 0.0103
Epoch 8/50
2736/2736 [==============================] - 4s 2ms/step - loss: 0.0102
Epoch 9/50
2736/2736 [==============================] - 5s 2ms/step - loss: 0.0101
Epoch 10/50
2736/2736 [==============================] - 4s 2ms/step - loss: 0.0102
Epoch 11/50
2736/2736 [==============================] - 5s 2ms/step - loss: 0.0101
Epoch 12/50
2736/2736 [==============================] - 4s 2ms/step - loss: 0.0102
E

## Question 9

### make predictions on train data
- use predict() method
  - batch_size: 1
- store predictions in a variable

In [98]:
pred = model.predict(X_train_rshp,batch_size=1)


In [99]:
pred

array([[0.6704707 ],
       [0.5994419 ],
       [0.625255  ],
       ...,
       [0.3698817 ],
       [0.28852665],
       [0.3535504 ]], dtype=float32)

### reset model state
- if you are using stateful=True in your model then you should call reset_states every time, when you want to make consecutive model calls independent.
- use reset_states() method

In [100]:
model.reset_states()

### make predictions on test data
- use predict() method
  - batch_size: 1
- store predictions in a variable

In [101]:
pred_test = model.predict(X_test_rshp,batch_size=1)

In [105]:
pred_test[:5]

array([[0.25681758],
       [0.26217058],
       [0.23924927],
       [0.24666142],
       [0.3290096 ]], dtype=float32)

## Question 10

### Inverse train predictions and true labels
- use your MinMaxSclarer object
- use inverse_transform() method

In [107]:
pred_inv = scaler.inverse_transform(pred)
y_train_inv = scaler.inverse_transform([y_train])

### Inverse test predictions and true labels
- use your MinMaxSclarer object
- use inverse_transform() method

In [108]:
pred_test_inv = scaler.inverse_transform(pred_test)
y_test_inv = scaler.inverse_transform([y_test])

In [113]:
y_test_inv

array([[ 5.8,  4.9,  5. ,  8.4, 12.3, 13. , 11.4,  6.8,  7.6, 12.4,  7.1,
         7.5, 10. ,  5.3,  6.3,  8. ,  8.3,  9.3,  9.5,  5.6,  7. ,  8.5,
         8.5,  8.2,  8.5,  9.6,  9.7,  7.1,  8.4,  9.2,  9.8,  8.1,  9.4,
        10. ,  5.1,  6.7,  6.9,  6.8,  8.6,  9.1,  3.9,  4.8,  8.4, 11.6,
        12.1, 12.4, 10. , 10.1,  9.7, 11.7,  7.9,  8.6,  7.7,  5.8,  8.7,
        10.6,  6.7,  8.8,  9.7,  9. , 11.8, 15.2, 10. , 10.5,  5.5,  9.4,
         8.8,  5.3, 13. , 15.2, 13.2, 11.5,  6.8,  4.7,  5.2,  6.8, 10.7,
        10.1, 10. ,  9.8,  5.5, 13.5, 16.6,  8.4,  8.2, 11.1, 10.8,  8.8,
        10.8,  8.7, 12.4,  9. , 13.5, 14.7, 10.9,  8.5,  6. , 12.7, 11.1,
         8.7, 12.3, 13.3,  5.6, 13.7,  8.5, 11.2,  8.7, 11.7, 12.5,  8.2,
        15.6, 10.3, 11.4,  9.7,  6.3, 14.3, 11.3,  7.3, 12.8, 11.9, 14.3,
        11.6, 13.2, 15.5, 14.1,  9.5,  7.2, 11.8, 16.8, 12.5,  9.4, 11.9,
        10.3, 16.9, 17.5,  7.5,  8.6, 11.1, 11.5, 10.7, 15.7, 12.8, 13. ,
        12.9, 14.3, 13.7, 12.1, 11.9, 

### Evaluate train predictions
- calculate RMSE
- RMSE = sqrt(mean_squared_error(true_labels, predicted_labels))
- use sqrt() and mean_squared_error() method
- print training RMSE

In [110]:
import math
from sklearn.metrics import mean_squared_error

In [114]:
train_score = np.sqrt(mean_squared_error(y_train_inv[0],pred_inv[:,0]))

In [119]:
print('Test score : ',train_score)

Test score :  2.6255031912974434


### Evaluate test predictions
- calculate RMSE
- RMSE = sqrt(mean_squared_error(true_labels, predicted_labels))
- use sqrt() and mean_squared_error() method
- print testing RMSE

In [116]:
test_score = np.sqrt(mean_squared_error(y_test_inv[0],pred_test_inv[:,0]))

In [120]:
print ('test score :',test_score)

test score : 2.5392651299654285


# Neural Machine Translation

- Translate a given sentence in one language to another desired language.

#### In this notebook, we aim to build a model which can translate German sentences to English.

## Dataset

Dataset is taken from http://www.manythings.org/anki/.

We are considering German – English deu-eng.zip file from the above mentioned website.

In the above zip file there is a file with name **`deu.txt`** that contains **152,820** pairs of English to German phrases, one pair per line with a tab separating the phrases.


For example,

The first 5 lines in deu.txt are as given below.

***
```
Hi.    Hallo!
Hi.    Grüß Gott!
Run!    Lauf!
Wow!    Potzdonner!
Wow!    Donnerwetter!
```
***


## Problem

### Given a sequence of words in German as input, predict the sequence of words in English.

### 1. Prepare Data


The preprocessing of the data involves:

1. Removing punctuation marks from the data.

2. Converting text corpus into lower case characters.

3. Split into Train and Test sets.

4. Shuffling the sentences.



The above tasks are done  and full dataset is given as **``english-german-both.pkl``** respectively.

Download dataset files from here: https://drive.google.com/open?id=1gWVk7SuuE93Cf_nT9Lb7GBCiwfAgdBiX




# Character level Machine Translation

## Initialize parameters
Run the below code to initialize the variables required for the model.

In [133]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

### Connect to google drive

In [121]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


### Give the path for the folder in which the dataset is present in google drive

In [141]:
project_path = "/content/drive/My Drive/assignment_11/"

### Change present working directory

In [142]:
import os
os.chdir(project_path)

## Load the pickle file (`english-german-both.pkl`) into a variable with name `dataset`
Run the below code to load the .pkl file.

In [143]:
import pickle

with open(project_path + 'english-german-both.pkl', 'rb') as f:
  dataset = pickle.load(f)

## Check the `dataset` variable at this step. It should be as given below

In [144]:
dataset

array([['stay with us', 'bleib bei uns'],
       ['she wants him', 'sie will ihn'],
       ['youre strong', 'du bist stark'],
       ...,
       ['i thought so', 'das dachte ich mir'],
       ['keep warm', 'haltet euch warm'],
       ['im sick', 'ich bin krank']], dtype='<U291')

## Feature set and target set division from the **dataset**

### Run the below code to divide the dataset into feature set(input) and target set(output). 

1. We are creating two lists for storing input sentences and output sentences separately. 
2. We are storing each character in a list from both input and target sets separately. 
3. Print and check `input_texts` and `target_texts`.
4. Print and check `input_characters` and `target_characters`.

In [148]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for line in dataset[: min(num_samples, len(dataset) - 1)]:
    input_text, target_text = line[0], line[1]
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

### Print input text

In [151]:
input_texts

['stay with us',
 'she wants him',
 'youre strong',
 'examine this',
 'heres my card',
 'tom burped',
 'it is no joke',
 'tom is a spy',
 'im a teenager',
 'im not crazy',
 'we cant do it',
 'i like sweets',
 'put it down',
 'tom looks fine',
 'i am hungarian',
 'youre dying',
 'go away',
 'tom denied this',
 'yes of course',
 'this is ugly',
 'tom cried again',
 'ill prove it',
 'do as he says',
 'please tell me',
 'he had ambition',
 'look here',
 'cuff him',
 'is it far away',
 'were cousins',
 'is tom dreaming',
 'i envy tom',
 'find a job',
 'im the expert',
 'just try it',
 'theyll find me',
 'a car hit tom',
 'have tom do it',
 'close your eyes',
 'come on in here',
 'i like this',
 'tom is history',
 'try it on',
 'ill sleep here',
 'does that help',
 'tom moved away',
 'well be ok',
 'im on my own',
 'toms out cold',
 'eat everything',
 'you said it',
 'he looked young',
 'dont insult me',
 'im upset',
 'you never ask',
 'i broke it',
 'theyre happy',
 'tom seemed sad',
 'tom 

### Print target text

In [152]:
print (target_texts)

['\tbleib bei uns\n', '\tsie will ihn\n', '\tdu bist stark\n', '\tuntersuchen sie das\n', '\thier ist meine karte\n', '\ttom stie auf\n', '\tdas ist kein witz\n', '\ttom ist ein spion\n', '\tich bin ein teenager\n', '\tich bin nicht verruckt\n', '\twir konnen das nicht\n', '\tich mag sues\n', '\tlegt es hin\n', '\ttom sieht gut aus\n', '\tich bin ungarin\n', '\tdu stirbst\n', '\tverkrumele dich\n', '\ttom hat das bestritten\n', '\tja naturlich\n', '\tdas ist grauslich\n', '\ttom hat wieder geweint\n', '\tich werde es beweisen\n', '\ttu was er sagt\n', '\tbitte sagt es mir\n', '\ter war strebsam\n', '\tschau her\n', '\tlegen sie ihm handschellen an\n', '\tist es weit weg\n', '\twir sind cousins\n', '\tist tom am traumen\n', '\tich beneide tom\n', '\tsuch dir einen job\n', '\tich bin der mann vom fach\n', '\tversuchs doch einfach\n', '\tsie werden mich finden\n', '\ttom wurde von einem auto angefahren\n', '\tlass es tom machen\n', '\tmacht die augen zu\n', '\tkomm herein\n', '\tich mag d

### Print input character

In [154]:
print (input_characters)

{'s', 'b', 'q', 'r', 'p', 'n', 'k', 'w', 'c', 'z', 'v', 'y', 'x', 'e', 'j', ' ', 'o', 'l', 'i', 'h', 't', 'g', 'm', 'u', 'd', 'a', 'f'}


### Print target character

In [138]:
print (target_characters)

{'s', 'b', 'q', 'r', 'p', 'n', 'k', 'w', 'c', 'z', 'v', 'y', 'e', 'x', 'j', ' ', 'o', '\t', 'l', 'i', 'h', '\n', 't', 'm', 'g', 'u', 'd', 'a', 'f'}


## Stats from the dataset

### Run the below code to check the stats from the dataset

In [155]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [156]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 9999
Number of unique input tokens: 27
Number of unique output tokens: 29
Max sequence length for inputs: 15
Max sequence length for outputs: 51


## Build character to index dictionary names `input_token_index` and `target_token_index` for input and target sets respectively.

In [164]:
input_token_index = dict([(char,i) for i,char in enumerate (input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate (target_characters)])

### Print input_index_token

In [165]:
input_token_index

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

### Print target_token_index

In [166]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28}

## Build Model
Initialize the required layers from keras

### Import libraries

In [167]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

### Run the below code to build one-hot vectors for the characters

In [168]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [169]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

### Build the encoder Model

Define an input sequence and process it.

Discard `encoder_outputs` and only keep the states.

In [172]:
decoder_inputs = Input(shape=(None,num_decoder_tokens))

decoder_lstm =LSTM(latent_dim,num_samples,batch_size,epochs,num_decoder_tokens,return_sequences=True)

TypeError: ignored

### Build the decoder Model

Set up the decoder, using `encoder_states` as initial state.

We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.

### Define Model

Define the model that will turn `encoder_input_data ` & ` decoder_input_data` into `decoder_target_data`

### Compile and fit the model

### Save the model

In [ ]:
# Save model
model.save('s2s.h5')

## Run the below code for inferencing the model

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

## Reverse-lookup token index to decode sequences back to something readable.

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
print(reverse_input_char_index)
print(reverse_target_char_index)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

## Run the below code for checking some outputs from the model.

In [ ]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)